# Potential Module

The `molpy.potential` package provides implementations of force-field potentials used for bonds, angles, dihedrals, and pairwise interactions. This notebook documents the concepts, the internal base-class/registration pattern, how to add custom force fields, naming conventions, and runnable examples that use the built-in `molpy.potential` implementations.

## Supported potential types

- Bond potentials: harmonic (e.g., `Harmonic`)
- Angle potentials: harmonic (angle)
- Dihedral potentials: cosine/poly-type (implementation dependent)
- Pair (non-bonded) potentials: Lennard–Jones (12-6), Coulomb (often with cutoff or switching)

Read the source in `src/molpy/potential` for the definitive API and available classes. This notebook now assumes the built-in potentials are available and imports them directly.


**Quick API reference**

Common classes and typical constructor parameters (check source for exact signatures):

- `Harmonic` (bond): `Harmonic(k: float, r0: float)` — k: force constant, r0: equilibrium bond length
- `Harmonic` (angle): `Harmonic(k: float, theta0: float)` — k: force constant, theta0: equilibrium angle
- `LJ126` (pair): `LJ126(epsilon: float, sigma: float)` — Lennard-Jones parameters
- `CoulCut` (pair): `CoulCut(cutoff: float = 10.0)` — Coulombic potential with cutoff

Most potential classes provide a `calc_energy(...)` method and optionally `calc_forces(...)` or `gradient(...)`. For exact names and signatures, consult `src/molpy/potential` or the generated API docs in `docs/api`.


**Base class and automatic registration**

The package uses a metaclass-based registration mechanism so potential classes are discovered automatically at import time. Key points:

- `Potential` (in `molpy.potential.base`) is the abstract base class. It declares the expected methods such as `calc_energy(...)` and `calc_forces(...)`.
- `KernelMeta` is a metaclass that runs when a potential class is defined. It reads the class attributes `type` (e.g. `"bond"`, `"pair"`) and `name` (e.g. `"harmonic"`) and inserts the class into `ForceField._kernel_registry`.
- `ForceField._kernel_registry` is a dict mapping type -> {name: class}. For example `ForceField._kernel_registry['bond']['harmonic']` points to the `Harmonic` class.

This automatic registration means you usually do not need to import or register classes manually — importing the module that defines the class is sufficient to register it.

In [ ]:
# Inspect the registry and instantiate a potential class by name
from molpy.core.forcefield import ForceField
import numpy as np

# Show registered kernel types (e.g. 'bond', 'pair')
print('kernel types:', list(ForceField._kernel_registry.keys()))

# Show registered implementations for bonds
print('bond implementations:', list(ForceField._kernel_registry.get('bond', {}).keys()))

# Look up the harmonic bond class and use it
HarmonicCls = ForceField._kernel_registry['bond']['harmonic']
print('Harmonic class:', HarmonicCls)

# Instantiate and compute energy for a single bond (two-atom system)
bond = HarmonicCls(k=100.0, r0=1.5)
bond_idx = np.array([[0, 1]])
bond_types = np.array([0])
coords = np.array([[0.0, 0.0, 0.0], [1.2, 0.0, 0.0]])
e = bond.calc_energy(coords, bond_idx, bond_types)
print('example bond energy:', e)

**How to add a custom potential (step-by-step)**

Follow these steps to add a new potential implementation to the package:

1. Create a new module under `src/molpy/potential/<category>/`, for example `src/molpy/potential/bond/my_custom.py`.
2. Implement a class that inherits the appropriate base (e.g. `BondPotential`, `AnglePotential`, or `PairPotential`).
3. Provide the class attributes `name` (string key) and `type` (the category, e.g. `'bond'`) so the metaclass can register it automatically.
4. Implement `calc_energy(...)` and `calc_forces(...)` with the same signatures used by other implementations in that category.
5. Add tests and update documentation. Importing the module is enough to register the implementation via `KernelMeta`.

Example file (`src/molpy/potential/bond/my_custom.py`):
```python
import numpy as np
from .base import BondPotential

class MyCustom(BondPotential):
    name = 'my_custom'
    type = 'bond'

    def __init__(self, k, r0):
        self.k = np.array(k, dtype=np.float64).reshape(-1,1)
        self.r0 = np.array(r0, dtype=np.float64).reshape(-1,1)

    def calc_energy(self, r, bond_idx, bond_types):
        # implement energy calculation similar to Harmonic
        dr = r[bond_idx[:,1]] - r[bond_idx[:,0]]
        dr_norm = np.linalg.norm(dr, axis=1, keepdims=True)
        energy = 0.5 * self.k[bond_types] * (dr_norm - self.r0[bond_types])**2
        return float(np.sum(energy))

    def calc_forces(self, r, bond_idx, bond_types):
        # implement forces consistent with calc_energy
        raise NotImplementedError
```

After you add the file, importing `molpy.potential.bond.my_custom` (or any module that imports it) will register `my_custom` in `ForceField._kernel_registry['bond']` automatically.

In [ ]:
# Demonstration: define a new BondPotential subclass dynamically and show automatic registration
from molpy.potential.bond.base import BondPotential
from molpy.core.forcefield import ForceField
import numpy as np

# Define a demo potential class dynamically; KernelMeta should register it
class DemoDynamic(BondPotential):
    name = 'demo_dynamic'
    type = 'bond'
    def __init__(self, k=10.0, r0=1.0):
        self.k = np.array(k, dtype=np.float64).reshape(-1,1)
        self.r0 = np.array(r0, dtype=np.float64).reshape(-1,1)
    def calc_energy(self, r, bond_idx, bond_types):
        dr = r[bond_idx[:,1]] - r[bond_idx[:,0]]
        dr_norm = np.linalg.norm(dr, axis=1, keepdims=True)
        energy = 0.5 * self.k[bond_types] * (dr_norm - self.r0[bond_types])**2
        return float(np.sum(energy))
    def calc_forces(self, r, bond_idx, bond_types):
        n_atoms = len(r)
        return np.zeros((n_atoms, 3))

# Check registry updated
print('demo_dynamic' in ForceField._kernel_registry.get('bond', {}))

# Instantiate and use the dynamic class
DemoCls = ForceField._kernel_registry['bond']['demo_dynamic']
demo = DemoCls(k=50.0, r0=1.2)
bond_idx = np.array([[0,1]])
bond_types = np.array([0])
coords = np.array([[0.0,0.0,0.0],[1.3,0.0,0.0]])
print('demo energy:', demo.calc_energy(coords, bond_idx, bond_types))

In [ ]:
# Harmonic bond example: use the built-in implementation from molpy
import numpy as np
from molpy.potential.bond import Harmonic

# Create a Harmonic bond potential with one type
bond = Harmonic(k=100.0, r0=1.5)

# We'll compute bond energy for a two-atom system where atom0 is at the origin
# and atom1 is placed at different distances along x. The Harmonic.calc_energy
# expects (r, bond_idx, bond_types).
bond_idx = np.array([[0, 1]])
bond_types = np.array([0])
for d in (1.2, 1.5, 1.8):
    coords = np.array([[0.0, 0.0, 0.0], [d, 0.0, 0.0]])
    e = bond.calc_energy(coords, bond_idx, bond_types)
    print(f'r={d:.2f} -> energy={e:.6f}')

TypeError: Harmonic.calc_energy() missing 2 required positional arguments: 'bond_idx' and 'bond_types'

**Lennard-Jones (LJ 12-6) example**

This example uses the built-in `LJ126` from `molpy.potential.pair` to compute pairwise LJ energies.

In [ ]:
import numpy as np
from molpy.potential.pair import LJ126

# Create LJ potential (one pair type)
lj = LJ126(epsilon=0.1, sigma=3.4)

# For pair potentials, the API expects pair displacement vectors and distances
# Prepare a single pair (atom0 at origin, atom1 at distance d along x)
pair_idx = np.array([[0, 1]])
pair_types = np.array([0])
for d in (2.5, 3.4, 4.5):
    coords = np.array([[0.0, 0.0, 0.0], [d, 0.0, 0.0]])
    dr = coords[pair_idx[:, 1]] - coords[pair_idx[:, 0]]  # shape (1,3)
    dr_norm = np.linalg.norm(dr, axis=1)  # shape (1,)
    e = lj.calc_energy(dr, dr_norm, pair_types)
    print(f'r={d:.2f} -> LJ energy={e:.6f}')

**Naming conventions**

- Module/file names: `snake_case` (e.g. `bond.py`, `lennard_jones.py`).
- Classes: `CamelCase` (e.g. `Harmonic`, `LJ126`, `CoulCut`).
- Class attributes: `name` (string key) should be lower-case with optional underscores (e.g. `'harmonic'`, `'lj126/cut'`).
- `type` attribute: one of the kernel categories (`'bond'`, `'angle'`, `'pair'`, `'dihedral'`, `'improper'`, etc.).
- Registry keys: the package registers by `ForceField._kernel_registry[type][name]` so keep `name` stable for backward compatibility.
- Parameter names: prefer `k`, `r0`, `theta0`, `epsilon`, `sigma`, `cutoff` and document expected units in the class docstring.
- Tests: include unit tests validating `calc_energy` and `calc_forces` consistency (finite-difference checks).

**Notes & references**

- Units: document expected units in the class docstring (energy: kJ/mol or kcal/mol; distance: Å or nm; angle: degrees or radians).
- For large systems consider cutoff, switching functions, and long-range corrections (Ewald/PME) for Coulomb interactions.
- For the authoritative code and the exact registration API, inspect `src/molpy/potential` in this repository.
- If you'd like visualizations (energy vs distance/angle) added to this notebook, I can add `matplotlib` cells next.

---
If you want me to also: 1) show the exact registry code from `src/molpy/potential`, 2) add dihedral/angle examples, or 3) include plots, tell me which one to do next.